In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(Seurat)
    library(Signac)
    library(EnsDb.Hsapiens.v86)
    library(dplyr)
    library(ggplot2)
    library(bedr)
    library(SeuratDisk)
    library(scales)
    library(reshape2)
    library(Hmisc)
    library(tidyr)
    library(tidyverse)
    library(crayon)
    library(readr)
    library(future)
}))

# Definitions etc

In [2]:
set.seed(1234)
options(digits=2)
stats <- c()

In [3]:
if(!exists("papermill_run")) {
    prj_name = "Screen1_66guides"
    data_subdir_suffixes = "L1,L2,L3,L4"     
    secondary_a_path = "/home/vsevim/prj/1012-ckd/S1/analysis/secondary/"
    save_seurat_h5 = "YES"
}

In [4]:
custom_theme <- theme(
  plot.title = element_text(size=16, hjust = 0.5), 
  legend.key.size = unit(0.7, "cm"), 
  legend.text = element_text(size = 14))

### Load Seurat file

In [5]:
integrated_h5_path <- "/home/vsevim/prj/1012-ckd/S1/analysis/secondary/integrated/seurat_objects/integrated.h5seurat"
seurat_combined    <- LoadH5Seurat(integrated_h5_path, verbose = FALSE)

Validating h5Seurat file

Warning message:
"Adding a command log without an assay associated with it"


Load THRESHOLDS.tsv

In [6]:
thr_f_name = paste0(secondary_a_path, "/integrated/THRESHOLDS.tsv")
df_thresholds = read.table(thr_f_name, header=T)
df_thresholds <- df_thresholds %>% pivot_wider(names_from = batch, values_from = threshold)
df_thresholds <- as.data.frame(df_thresholds)

# Differential expression

## Find guide+ (perturbed) and guide- (unperturbed) cells for each guide

In [7]:
libraries = unique(seurat_combined$library)
seurat_libs = list()
for(i in seq_along(libraries)){ 
    lib = libraries[i]
    seurat_libs[[i]] = subset(seurat_combined, subset = library == lib)
}
names(seurat_libs) <- libraries

In [15]:
perturbed_cells_by_guide = list()

for(i in 1:nrow(df_thresholds)){  
    perturbed_cells_in_all_libs = list()
    guide = df_thresholds$guide[i]
    # Loop over libraries
    for(lib in libraries){        
        seurat_lib = seurat_libs[[lib]]
        threshold = df_thresholds[i, lib]        
        #cat(blue(guide, lib, threshold, "\n"))
        cells_in_lib = Cells(seurat_lib)        
        sgrna_counts = seurat_lib[['sgRNA']]@counts
        select_perturbed = sgrna_counts[guide, cells_in_lib] >= threshold
        perturbed_cells_in_library = cells_in_lib[select_perturbed]
        #cat(length(cells_in_lib), "in", lib, guide, length(perturbed_cells_in_library), "cells >", threshold, "\n")        
        perturbed_cells_in_all_libs = append(perturbed_cells_in_all_libs, perturbed_cells_in_library)
    }
    perturbed_cells_by_guide[[i]] = perturbed_cells_in_all_libs
}
names(perturbed_cells_by_guide) <- df_thresholds$guide

In [ ]:
dummy_select = sgrna_counts['CD13-1a', cells_ids_in_lib] >= 13
sum(dummy_select)
min(sgrna_counts['CD13-1a', dummy_select])

In [16]:
length(perturbed_cells_by_guide[[3]])

[1] 1060

In [17]:
df_thresholds$guide[grepl("NT" , df_thresholds$guide)]

[1] "NTCE1-1a" "NTCE3-1a" "NTCE3-1b" "NTCE4-1a" "NTCE4-1b"

## Run DE

In [74]:
seurat_rna = CreateSeuratObject(seurat_combined[['RNA']])
seurat_rna <- NormalizeData(seurat_rna)
seurat_rna@meta.data <- seurat_combined@meta.data

In [89]:
guides = c('PLIN3-1a', 'PLIN3-1b')

all_cells = Cells(seurat_rna)
perturbed_cells = unlist(perturbed_cells_by_guide[[guide]])




unperturbed_cells = unlist(setdiff(all_cells, perturbed_cells))
Idents(seurat_rna) <- "guide_negative"
seurat_rna <- SetIdent(seurat_rna, cells = perturbed_cells, value = "guide_positive") 

length(perturbed_cells)
length(unperturbed_cells)

markers <- FindMarkers(
    object = seurat_rna,
    slot = "data",
    ident.1 = 'guide_positive',
    ident.2 = 'guide_negative',
    test.use = "MAST",
    latent.vars = 'library',
    verbose = F
)

[1] 1161

[1] 23402


 Completed [============================================] 100% with 0 failures
                                                                              


Done!

Combining coefficients and standard errors

Calculating log-fold changes

Calculating likelihood ratio tests

Refitting on reduced model...


Done!



In [90]:
markers

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PLIN3,1.7e-78,-0.31,0.69,0.81,6.3e-74
AL138828.1,6.8e-12,0.33,0.54,0.45,2.5e-07
HMOX1,8.5e-10,0.26,0.77,0.70,3.1e-05


In [82]:
markers

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SQSTM1,4.6e-38,0.29,1.00,0.99,1.7e-33
HMOX1,1.1e-37,0.43,0.78,0.70,3.9e-33
AL357507.1,7.6e-30,0.28,0.31,0.20,2.8e-25
AL138828.1,9.3e-21,0.32,0.54,0.44,3.4e-16
PLIN2,2.1e-18,0.28,0.94,0.92,7.5e-14
TUBA1B,1.8e-15,-0.25,0.87,0.87,6.8e-11
